<a href="https://colab.research.google.com/github/virajvaidya/SRC_ANU_Sampling_1/blob/main/HomeworkW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import statistics as st
import itertools as it
import scipy.stats as stats
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.stats import t

In [9]:
pop = pd.read_excel('/content/drive/MyDrive/Sampling I - HW6 data.xls')

In [10]:
pop.head(10)

,STRATUM,CLUSTER_ID2,ELEMENT,BMI,EVERPREG,PROB
0,1,1,1,26,1,0.004167
1,1,1,2,42,1,0.004167
2,1,1,3,20,1,0.004167
3,1,1,4,26,1,0.004167
4,1,2,5,19,1,0.004545
5,1,2,6,22,0,0.004545
6,1,2,7,22,0,0.004545
7,1,2,8,49,1,0.004545
8,2,3,9,30,1,0.014286
9,2,3,10,20,0,0.014286


## Q 1 a Compute the sampling weight for each selected woman in your sample

In [16]:
def compute_sampling_weights(file_path):
    df = pop

    df['sampling_weight'] = 1 / df['PROB']

    return df[['STRATUM', 'CLUSTER_ID2', 'ELEMENT', 'sampling_weight']]

In [17]:
weights_pop = compute_sampling_weights(pop)
print(weights_pop)

    STRATUM  CLUSTER_ID2  ELEMENT  sampling_weight
0         1            1        1          240.000
1         1            1        2          240.000
2         1            1        3          240.000
3         1            1        4          240.000
4         1            2        5          220.000
5         1            2        6          220.000
6         1            2        7          220.000
7         1            2        8          220.000
8         2            3        9           70.000
9         2            3       10           70.000
10        2            3       11           70.000
11        2            3       12           70.000
12        2            4       13          465.000
13        2            4       14          465.000
14        2            4       15          465.000
15        2            4       16          465.000
16        3            5       17          319.000
17        3            5       18          319.000
18        3            5       



---



## Q1 b What is the estimated increase in variance due to these weights? What is the assumption you have to rely on to compute this “weighting effect”?

In [18]:
def compute_weighting_effect(file_path):

    df = pop.copy()

    df['sampling_weight'] = 1 / df['PROB']

    # Calculating Coefficient of Variation Squared (CV^2)
    mean_w = df['sampling_weight'].mean()
    var_w = df['sampling_weight'].var()
    cv_squared = var_w / (mean_w ** 2)

    # Weighting effect
    weighting_effect = 1 + cv_squared

    return weighting_effect

In [20]:
we = compute_weighting_effect(pop)
print(we)

1.257268559903654


##The weghitng effect is 1.257268. The assumption we have to rely on is that of independently random sampling between each stratum.



---



##Q 1 c Complete the table with weighted cluster totals (*Wha* is the sum of theweights in the cluster, *Uha* is the weighted cluster total for BMI and *Vha* si the weighted cluster total for the number of women who have ever been pregnant in the cluster )

In [25]:
def compute_weighted_cluster_totals(pop):

    required_cols = ['STRATUM', 'CLUSTER_ID2', 'BMI', 'EVERPREG', 'PROB']
    for col in required_cols:
        if col not in pop.columns:
            raise ValueError(f"Missing required column: '{col}'")

    #Sampling Weights
    pop['sampling_weight'] = 1 / pop['PROB']

    # Assign Cluster Index α (1, 2 within each Stratum)
    pop['α'] = pop.groupby('STRATUM')['CLUSTER_ID2'].rank(method='dense').astype(int)

    #Weighted Totals Per Cluster
    grouped = pop.groupby(['STRATUM', 'α', 'CLUSTER_ID2']).agg(
        w_hα=('sampling_weight', 'sum'),   # Sum of weights in the cluster
        u_hα=('BMI', lambda x: (x * pop.loc[x.index, 'sampling_weight']).sum()),  # Weighted BMI total
        v_hα=('EVERPREG', lambda x: (x * pop.loc[x.index, 'sampling_weight']).sum())  # Weighted EVERPREG total
    ).reset_index()

    grouped.rename(columns={'STRATUM': 'h', 'CLUSTER_ID2': 'CLUSTER_ID'}, inplace=True)


    total_row = grouped[['w_hα', 'u_hα', 'v_hα']].sum()
    total_row_df = pd.DataFrame([['Total', '', '', *total_row]], columns=grouped.columns)
    final_table = pd.concat([grouped, total_row_df], ignore_index=True)
    return final_table


table = compute_weighted_cluster_totals(pop)


from google.colab import data_table
data_table.DataTable(table)



,h,α,CLUSTER_ID,w_hα,u_hα,v_hα
0,1,1,1,960.0,27360.000,960.00
1,1,2,2,880.0,24640.000,440.00
2,2,1,3,280.0,6650.000,140.00
3,2,2,4,1860.0,59055.000,1395.00
4,3,1,5,1276.0,35090.000,1276.00
5,3,2,6,768.5,19788.875,384.25
6,4,1,7,876.0,28251.000,876.00
7,4,2,8,732.0,20496.000,366.00
8,5,1,9,486.0,13972.500,486.00
9,5,2,10,459.0,13196.250,344.25




---



## Q1 d Calculate the mean BMI, SE and 99% CI

In [29]:

def estimate_mean_bmi_fixed(pop, N=9300):
    #sampling weights
    pop['sampling_weight'] = 1 / pop['PROB']

    #weighted mean BMI
    mean_bmi = np.average(pop['BMI'], weights=pop['sampling_weight'])

    # weighted variance
    mean_diff_sq = (pop['BMI'] - mean_bmi) ** 2
    weighted_var = np.sum(pop['sampling_weight'] * mean_diff_sq) / np.sum(pop['sampling_weight'])

    # Compute standard error (SE)
    se_bmi = np.sqrt(weighted_var / len(pop))

    # Finite Population Correction (FPC)
    if N:
        fpc = np.sqrt((N - len(pop)) / (N - 1))
        se_bmi *= fpc

    # 99% Confidence Interval (CI)
    z_score = stats.norm.ppf(0.995)  # 99% CI -> 0.5% in each tail
    ci_lower = mean_bmi - z_score * se_bmi
    ci_upper = mean_bmi + z_score * se_bmi

    return mean_bmi, se_bmi, (ci_lower, ci_upper)

# Compute results with FPC
mean_bmi, se_bmi, ci_bmi = estimate_mean_bmi_fixed(pop)

# Print results
print(f"Estimated Mean BMI: {mean_bmi:.4f}")
print(f"Standard Error (SE): {se_bmi:.4f}")
print(f"99% Confidence Interval: ({ci_bmi[0]:.4f}, {ci_bmi[1]:.4f})")


Estimated Mean BMI: 28.9711
Standard Error (SE): 1.2342
99% Confidence Interval: (25.7919, 32.1503)


## Estimated Mean BMI: 28.9711
## Standard Error (SE): 1.2342
## 99% Confidence Interval: (Lower 25.7919, Upper  32.1503)



---



 ## Q1 e Compute the proportion of women who have ever been pregnant and its standard error.

In [30]:
def estimate_proportion_everpreg(pop):

    pop['sampling_weight'] = 1 / pop['PROB']

    # Weighted proportion of women who have ever been pregnant
    p_hat = np.average(pop['EVERPREG'], weights=pop['sampling_weight'])

    # Variance calculation
    var_p = np.sum(pop['sampling_weight']**2 * (pop['EVERPREG'] - p_hat)**2) / (np.sum(pop['sampling_weight'])**2)

    # Standard Error (SE)
    se_p = np.sqrt(var_p)

    return p_hat, se_p

p_hat, se_p = estimate_proportion_everpreg(pop)

print(f"Estimated Proportion of Ever Pregnant Women: {p_hat:.4f}")
print(f"Standard Error (SE): {se_p:.4f}")


Estimated Proportion of Ever Pregnant Women: 0.7773
Standard Error (SE): 0.0717


##Estimated Proportion of Ever Pregnant Women: 0.7773
Standard Error (SE): 0.0717



---



## Compute the design effect and roh for the mean in the above

In [37]:
import numpy as np
import pandas as pd

def deffrho(pop, b=4):

    pop['sampling_weight'] = 1 / pop['PROB']

    # weighted mean BMI
    mean_bmi = np.average(pop['BMI'], weights=pop['sampling_weight'])

    # variance under simple random sampling (SRS)
    var_srs = np.var(pop['BMI'], ddof=1) / len(pop)

    # CLuster meams
    cluster_means = pop.groupby('CLUSTER_ID2').apply(
        lambda x: np.average(x['BMI'], weights=x['sampling_weight'])
    )

    #between-cluster variance
    var_between = np.var(cluster_means, ddof=1)
    #within cluster variance
    var_within = pop.groupby('CLUSTER_ID2').apply(
        lambda x: np.var(x['BMI'], ddof=1)
    ).mean()

    # Deff
    Deff = 1 + ((b * var_between) / var_within)

    # Rho
    roh = var_between / (var_within + var_between)

    return Deff, roh


Deff, roh = deffrho(pop)


print(f"Design Effect (Deff): {Deff:.4f}")
print(f"Intraclass Correlation (Rho, ρ): {roh:.4f}")


Design Effect (Deff): 1.3458
Intraclass Correlation (Rho, ρ): 0.0796


<ipython-input-37-bf31eb8d9404>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_means = pop.groupby('CLUSTER_ID2').apply(
<ipython-input-37-bf31eb8d9404>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  var_within = pop.groupby('CLUSTER_ID2').apply(


## Design Effect (Deff): 1.3458
## (Rho, ρ): 0.0796



---



## Q1 g Is the Taylor series approximation adequate for this problem?

In [40]:
# Coefficient of variation of weights
cv_weights = pop['sampling_weight'].std() / pop['sampling_weight'].mean()


print(f"Coefficient of Variation (CV) of Weights: {cv_weights:.4f}")

# Check if Taylor approximation is adequate - using a rule of thumb , not super accurate
if cv_weights < 0.3:
    print("✅ Taylor series approximation is adequate for this problem.")
else:
    print("❌ Taylor series approximation may underestimate variance.")


Coefficient of Variation (CV) of Weights: 0.5072
❌ Taylor series approximation may underestimate variance.


## Taylor series approximation may underestimate variance, and may not be enough for this problem.



---



## Q 2 a ~12:00
## Q 2 b 9.5



---

